In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train_dir = '/Users/jeffreyscruggs/Desktop/DataSets/train'
validation_dir = '/Users/jeffreyscruggs/Desktop/DataSets/val'

In [5]:
train_parasitized_dir = os.path.join(train_dir, 'Parasitized') # directory with our training elephant pictures
train_uninfected_dir = os.path.join(train_dir, 'Uninfected') # directory with our training tiger pictures
validation_parasitized_dir = os.path.join(validation_dir, 'Parasitized') # directory with our validation elephant pictures
validation_uninfected_dir = os.path.join(validation_dir, 'Uninfected') # directory with our validation tiger pictures

In [6]:
num_parasitized_tr = len(os.listdir(train_parasitized_dir))
num_uninfected_tr = len(os.listdir(train_uninfected_dir))
num_parasitized_val = len(os.listdir(validation_parasitized_dir))
num_uninfected_val = len(os.listdir(validation_uninfected_dir))
total_train = num_parasitized_tr + num_uninfected_tr
total_val = num_parasitized_val + num_uninfected_val
print('total training parasitized images:', num_parasitized_tr)
print('total training uninfected images:', num_uninfected_tr)

print('total validation parasitized images:', num_parasitized_val)
print('total validation uninfected images:', num_uninfected_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training parasitized images: 11023
total training uninfected images: 11023
total validation parasitized images: 2756
total validation uninfected images: 2756
--
Total training images: 22046
Total validation images: 5512


In [7]:
batch_size = 128
epochs = 5
IMG_HEIGHT = 64
IMG_WIDTH = 64

In [8]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [9]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
 directory=train_dir,
 shuffle=True,
 target_size=(IMG_HEIGHT, IMG_WIDTH),
 class_mode='binary')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
 directory=validation_dir,
 target_size=(IMG_HEIGHT, IMG_WIDTH),
 class_mode='binary')

Found 22044 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


In [10]:
model = Sequential([
 Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
 MaxPooling2D(),
 Conv2D(32, 3, padding='same', activation='relu'),
 MaxPooling2D(),
 Conv2D(64, 3, padding='same', activation='relu'),
 MaxPooling2D(),
 Flatten(),
 Dense(512, activation='relu'),
 Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [13]:
history = model.fit_generator(
 train_data_gen,
 steps_per_epoch= total_train // batch_size,
 epochs=epochs,
 validation_data= val_data_gen,
 validation_steps= total_val // batch_size
)

Epoch 1/5
172/172 [==============================] - 97s 564ms/step - loss: 0.6540 - accuracy: 0.6058 - val_loss: 0.5219 - val_accuracy: 0.7525
Epoch 2/5
172/172 [==============================] - 96s 559ms/step - loss: 0.3818 - accuracy: 0.8264 - val_loss: 0.1889 - val_accuracy: 0.9331
Epoch 3/5
172/172 [==============================] - 97s 566ms/step - loss: 0.1646 - accuracy: 0.9426 - val_loss: 0.1516 - val_accuracy: 0.9477
Epoch 4/5
172/172 [==============================] - 96s 558ms/step - loss: 0.1358 - accuracy: 0.9545 - val_loss: 0.1385 - val_accuracy: 0.9535
Epoch 5/5
172/172 [==============================] - 95s 549ms/step - loss: 0.1240 - accuracy: 0.9585 - val_loss: 0.1471 - val_accuracy: 0.9500


In [14]:
tf.__version__

'2.0.0'

In [17]:
model.save('malariaClasifier2')

INFO:tensorflow:Assets written to: malariaClasifier2/assets


In [18]:
print(train_data_gen.class_indices)

{'Parasitized': 0, 'Uninfected': 1}
